In [1]:
# import os

# # Set R environment variables using the conda environment path
# r_home = '/sfs/gpfs/tardis/home/jq2uw/llm_nicu_vitalsigns/clip_env/lib/R'
# os.environ['R_HOME'] = r_home
# os.environ['R_LIBS'] = f"{r_home}/library"
# os.environ['R_LIBS_USER'] = os.path.expanduser('~/R/goolf/4.3')
# os.environ['LD_LIBRARY_PATH'] = f"{r_home}/lib:" + os.environ.get('LD_LIBRARY_PATH', '')


In [2]:
from data import *
from config import *
from classifer import *
from classifier_shapelet import *
from describer import *
print("using device: ", device)
random_state = 333
import pandas as pd
from sklearn.model_selection import train_test_split

# for debugging
# import importlib
# import models
# importlib.reload(models)


Random seed set to 333
using device:  cpu


In [3]:
# -----------------Train Data-----------------
df = pd.read_excel('../../data/PAS Challenge HR Data.xlsx', engine="calamine")
df.columns = df.columns.astype(str)
df_y = pd.read_excel('../../data/PAS Challenge Outcome Data.xlsx', engine="calamine")[['VitalID', 'DiedNICU', 'DeathAge']]
df_demo = pd.read_excel('../../data/PAS Challenge Demographic Data.xlsx', engine="calamine")
df_x = pd.read_excel('../../data/PAS Challenge Model Data.xlsx', engine="calamine")
df = df.merge(df_x[['VitalID', 'VitalTime', 'Age']], on=['VitalID', 'VitalTime'], how='left')
df = label_death7d(df, df_y, id_col='VitalID')
df = df.merge(df_demo, on='VitalID', how='left')
df_desc = generate_descriptions_parallel(ts_df = df.loc[:, '1':'300'], id_df = df.loc[:, ['VitalID', 'VitalTime']])
df = df.merge(df_desc, on=['VitalID', 'VitalTime'], how='left')
df = text_gen_input_column(df, config_dict['text_config'])
df['rowid'] = df.index.to_series() 
df_train = df

# -----------------Test Data-----------------
df_y_test = pd.read_excel('../../data/Test Data/Test Demographic Key.xlsx', sheet_name=0, engine="calamine")
df_test = pd.read_excel('../../data/Test Data/Test HR Data.xlsx', sheet_name=0, engine="calamine") # test hr with description
df_test.columns = df_test.columns.astype(str)
df_test = label_death7d(df_test, df_y_test, id_col='TestID')
df_demo_test = pd.read_excel('../../data/Test Data/Test Demographic Data.xlsx', sheet_name=0, engine="calamine")
df_test = df_test.merge(df_demo_test, on='TestID', how='left')
df_test['rowid'] = df_test.index.to_series()
df_test['VitalTime'] = df_test['Age']*24*60*60 # convert to second since birth
df_test['VitalTime'] = df_test['VitalTime'].astype(int)
rename_dict = {'TestID': 'VitalID'}
df_test = df_test.rename(columns=rename_dict)

df_desc_test = generate_descriptions_parallel(ts_df = df_test.loc[:, '1':'300'], id_df = df_test.loc[:, ['VitalID', 'VitalTime']])
df_test = df_test.merge(df_desc_test, on=['VitalID', 'VitalTime'], how='left')
df_test = text_gen_input_column(df_test, config_dict['text_config'])
df_test_org = df_test[df.columns]
df_test, df_leftout = train_test_split(df_test_org, test_size=0.5, stratify=df_test_org['cl_event'], random_state=random_state) 


Sample of patients with positive labels:
VitalID
1018    8
5170    8
1464    8
2361    8
2791    8
dtype: int64


[Parallel(n_jobs=9)]: Using backend LokyBackend with 9 concurrent workers.
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    4.5s
[Parallel(n_jobs=9)]: Done 5138 tasks      | elapsed:    5.4s
[Parallel(n_jobs=9)]: Done 63704 tasks      | elapsed:   12.6s
[Parallel(n_jobs=9)]: Done 65353 out of 65353 | elapsed:   12.7s finished


This infant will survive.  This infant has gestational age 24 weeks. Birth weight is 360 grams.  No Bradycardia events.   Very low amount of consecutive increases. 

Sample of patients with positive labels:
TestID
817     8
1903    8
801     8
508     8
2518    8
dtype: int64


[Parallel(n_jobs=9)]: Using backend LokyBackend with 9 concurrent workers.
[Parallel(n_jobs=9)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 14318 tasks      | elapsed:    2.2s
[Parallel(n_jobs=9)]: Done 61182 tasks      | elapsed:    7.7s
[Parallel(n_jobs=9)]: Done 61570 out of 61570 | elapsed:    7.7s finished


This infant will survive.  This infant has gestational age 33 weeks. Birth weight is 2630 grams.  No Bradycardia events.  It shows high variability.  Low amount of consecutive increases. 


In [4]:
txt_ls_org = ['This infant will die in 7 days. ', 'This infant will survive. ']
y_col = 'cl_event'
# # ---- downsample negative class(es) ----
# # sample 5000 rows from df_test[df_test[y_col]==txt_ls_org[1]], without replacement
# neg_sample_size = 1000
# df_test_downsampled = df_test[df_test[y_col]==txt_ls_org[1]].sample(n=neg_sample_size, replace=False, random_state=random_state)
# df_test = pd.concat([df_test[df_test[y_col]==txt_ls_org[0]], df_test_downsampled])
# df_train_downsampled = df_train[df_train[y_col]==txt_ls_org[1]].sample(n=neg_sample_size, replace=False, random_state=random_state)
# df_train = pd.concat([df_train[df_train[y_col]==txt_ls_org[0]], df_train_downsampled])

# ---- for testing only ----
# sample 50 rows from df_train[df_train[y_col]==txt_ls_org[1]], without replacement
# sample 50 rows from df_train[df_train[y_col]==txt_ls_org[0]], without replacement
df_test_neg = df_test[df_test[y_col]==txt_ls_org[1]].sample(n=50, replace=False, random_state=random_state)
df_test_pos = df_test[df_test[y_col]==txt_ls_org[0]].sample(n=50, replace=False, random_state=random_state)
df_test = pd.concat([df_test_neg, df_test_pos])
df_train_neg = df_train[df_train[y_col]==txt_ls_org[1]].sample(n=50, replace=False, random_state=random_state)
df_train_pos = df_train[df_train[y_col]==txt_ls_org[0]].sample(n=50, replace=False, random_state=random_state)
df_train = pd.concat([df_train_neg, df_train_pos])


# ---- augment + balance train data----
target_event_rate = len(df_test[df_test[y_col]==txt_ls_org[0]])/len(df_test)
max_size = int(target_event_rate*len(df_train))
if config_dict['ts_aug']:
    df_train = augment_balance_data(df_train, 
                                    txt_ls_org, 
                                    y_col, 
                                    config_dict, 
                                    pretrained_model_path='./pretrained/hr_vae_linear_medium.pth', 
                                    K=10,
                                    max_size=max_size)


# ---- block or not ----
# important for generating labels for block target
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
if not config_dict['block_target']:
    df_train['label'] = df_train.index.to_series()
    df_test['label'] = df_test.index.to_series()
else:
    df_train['label'] = df_train['rowid'].astype(int)
    df_test['label'] = df_test['rowid'].astype(int)

def get_y_true_and_ts_df(df_new, txt_ls_org):
    df_new_y = pd.get_dummies(df_new['cl_event'])
    df_new_y = df_new_y[txt_ls_org]
    y_true = torch.tensor(df_new_y.values)
    ts_df = df_new.loc[:,'1':'300']
    return y_true, ts_df


# ---- prepare for binary classification evaluation ----
y_true_train, ts_df_train = get_y_true_and_ts_df(df_train, txt_ls_org)     # create y_true_train and ts_df_train
y_true_test, ts_df_test = get_y_true_and_ts_df(df_test, txt_ls_org)     # create y_true_test and ts_df_test
print(ts_df_train.shape)
print(ts_df_test.shape)


y_true_train = y_true_train.float()
y_true_test = y_true_test.float() 
train_loader = prepare_basedata(ts_df_train, y_true_train)
test_loader = prepare_basedata(ts_df_test, y_true_test)

# get number of positives in y_true_train
pos_count = (y_true_train[:,0] == 1).sum().item()
print(f"Number of positives in y_true_train: {pos_count}")

# get number of positives in y_true_test
pos_count = (y_true_test[:,0] == 1).sum().item()
print(f"Number of positives in y_true_test: {pos_count}")



X_train = np.array(ts_df_train) # nobs x 300
y_train = np.array(y_true_train[:,0])# nobs ,
X_test = np.array(ts_df_test) # nobs x 300
y_test = np.array(y_true_test[:,0])# nobs ,


(100, 300)
(100, 300)
Number of positives in y_true_train: 50
Number of positives in y_true_test: 50


# shapelet models to predict clinical outcomes end2end

In [5]:

# # Initialize classifier
# clf = ShapeletClassifier(
#     min_length=15, # 0.5 minute worth of data points
#     max_length=60, # 2 minute worth of data points
#     n_shapelets=5
# )

# # Fit classifier
# clf.fit(X_train, y_train) 

# # Make predictions
# y_pred = clf.predict(X_test)

# # Get performance metrics
# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)

# # Get learned shapelets
# shapelets = clf.get_shapelets()

# # Visualize shapelets
# import matplotlib.pyplot as plt

# def plot_shapelets(shapelets, title="Discovered Shapelets"):
#     fig, axes = plt.subplots(len(shapelets), 1, figsize=(10, 2*len(shapelets)))
#     for i, (shapelet, score) in enumerate(shapelets):
#         axes[i].plot(shapelet)
#         axes[i].set_title(f"Shapelet {i+1}, Information Gain: {score:.4f}")
#     plt.tight_layout()
#     plt.show()

# plot_shapelets(shapelets)

In [9]:

clf = FastShapeletClassifier(
    min_length=110,    # 1 minute
    max_length=115,   # 10 minutes
    n_shapelets=5,
    n_jobs=4#
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


# Get performance metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Get learned shapelets
shapelets = clf.get_shapelets()

# Visualize shapelets

def plot_shapelets(shapelets, title="Discovered Shapelets"):
    fig, axes = plt.subplots(len(shapelets), 1, figsize=(10, 2*len(shapelets)))
    for i, (shapelet, score) in enumerate(shapelets):
        axes[i].plot(shapelet)
        axes[i].set_title(f"Shapelet {i+1}, Information Gain: {score:.4f}")
    plt.tight_layout()
    plt.show()

plot_shapelets(shapelets)



Extracting candidates...
Evaluating 19100 shapelets in parallel...


Finding shapelets:  63%|██████▎   | 12087/19100 [03:10<01:49, 63.99it/s]